In [1]:
from gpt_index import LLMPredictor
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

In [2]:
DEFAULT_PROMPT = """
文脈情報は以下です。
---
{context_str}
---
事前知識ではなく、文脈情報を参考に質問に答えてください。：{query_str}
"""

REFINE_PROMPT = """
質問は以下です。：{query_str}
すでに答えの候補があります。：{existing_answer}
必要な場合のみ、以下の文脈情報を使ってこの答えを改良することができます。
---
{context_msg}
---
この文脈情報により、元の答えを改良して質問に答えてください。
文脈情報が有用でない場合は元の答えをそのまま返してください。
"""

In [3]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
qa_model = AutoModelForCausalLM.from_pretrained("oshizo/qa-refine-japanese-gpt-1b")

if torch.cuda.is_available():
    qa_model = qa_model.to("cuda")


In [4]:
def generate(prompt):

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    n = len(token_ids[0])

    with torch.no_grad():
        output_ids = qa_model.generate(
            token_ids.to(qa_model.device),
            max_length=n+100,
            min_length=n+2,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids.tolist()[0][n:])
    return output.replace("</s>", "")

In [5]:
class CustomLLM(LLM):
        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return generate(prompt)
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name":"custom"}
llm = CustomLLM()
llm_predictor = LLMPredictor(llm)

In [6]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from gpt_index import LangchainEmbedding

# load in HF embedding model from langchain
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())

In [7]:
from sentence_transformers import SentenceTransformer
embed_model._langchain_embedding.client = SentenceTransformer("oshizo/sbert-jsnli-luke-japanese-base-lite")
embed_model._langchain_embedding.client.max_seq_length = 256

In [8]:
import urllib
import json
url = "https://ja.wikipedia.org/wiki/%E3%81%BC%E3%81%A3%E3%81%A1%E3%83%BB%E3%81%96%E3%83%BB%E3%82%8D%E3%81%A3%E3%81%8F!?action=cirrusdump"
with urllib.request.urlopen(url) as f:
    data = f.read()
text = json.loads(data)[0]["_source"]["text"]

In [ ]:
from gpt_index import GPTSimpleVectorIndex
from gpt_index.readers.schema.base import Document
documents = []
for i in range(0, len(text), 200):
    documents.append(Document(text[i:i+200]))
    if i != 0:
        documents.append(Document(text[i-100:i+100]))
# インデックスの作成
index = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor, embed_model=embed_model)

In [12]:
len(documents)

379

In [13]:
from gpt_index.prompts.base import Prompt
from gpt_index.prompts.prompts import RefinePrompt, QuestionAnswerPrompt
refine_prompt = RefinePrompt(REFINE_PROMPT)
default_prompt = QuestionAnswerPrompt(DEFAULT_PROMPT)

In [22]:
response = index.query(
    "虹夏ちゃんのお姉さんの仕事は？", 
    mode="embedding", 
    verbose=True, 
    embed_model=embed_model,
    text_qa_template=default_prompt,
    refine_template=refine_prompt,
    similarity_top_k=2
)
response.response

> Top 2 nodes:
> [Node 861c16bc-553f-47e7-a1e0-55cd5fda9bb6] 地知 星歌（いじち せいか） 声 - 内田真礼 虹夏の姉で、ライブハウス「STARRY」の店長。クリスマスイブが誕生日で、作中1年目に30歳を迎えた。かつては自身もバンドマン（ギタリスト）で、そ...
> [Node 6de817d0-b70a-417f-aae6-a02c0b233c64]  鈴代紗弓 誕生日：5月29日 / 血液型：A型 ドラム担当。下北沢高校2年→3年。明るく世話焼きで、バンドのまとめ役。水玉のリボンがトレードマークで、たいてい身体のどこかに身につけている。姉の...
> Searching in chunk: 地知 星歌（いじち せいか） 声 - 内田真礼 虹夏の姉で、ライブハウス「STARRY」の店長...
> Searching in chunk:  鈴代紗弓 誕生日：5月29日 / 血液型：A型 ドラム担当。下北沢高校2年→3年。明るく世話...
> Initial response: ライブハウス「STARRY」の店長
> Refine context:  鈴代紗弓 誕生日：5月29日 / 血液型：A型 ドラム担当。下北沢高校2年→3年。明るく世話...
> Refined response: ライブハウス「STARRY」の店長
> [query] Total LLM token usage: 931 tokens
> [query] Total embedding token usage: 26 tokens


'ライブハウス「STARRY」の店長'

In [23]:
response = index.query(
    "後藤ひとりがギターに熱中するようになった理由になった出来事は？", 
    mode="embedding", 
    verbose=True, 
    embed_model=embed_model,
    text_qa_template=default_prompt,
    refine_template=refine_prompt,
    similarity_top_k=2
)
response.response

> Top 2 nodes:
> [Node a81cd7ff-99b6-42a9-bc72-5e35d5c0c1f5] レックスを持っていたところ、中学1年の時に、暗い学生時代から一転してスターとなったバンドマンのインタビューを目にしたことで、父親から借りたギターに没頭する。毎日6時間以上の練習を約3年間欠かさず...
> [Node 4b2fd373-68fe-4830-9161-246f88257149] である所謂「陽キャラ」および「パリピ」に強い偏見を抱いており、虹夏や喜多らを自分にとってのヒール役に見立てた妄想をすることがある。 先述の性格に加え、運動、勉強など取り柄と言えるものがないという...
> Searching in chunk: レックスを持っていたところ、中学1年の時に、暗い学生時代から一転してスターとなったバンドマンの...
> Searching in chunk: である所謂「陽キャラ」および「パリピ」に強い偏見を抱いており、虹夏や喜多らを自分にとってのヒー...
> Initial response: 暗い学生時代から一転してスターとなったバンドマンのインタビュー
> Refine context: である所謂「陽キャラ」および「パリピ」に強い偏見を抱いており、虹夏や喜多らを自分にとってのヒー...
> Refined response: 暗い学生時代から一転してスターとなったバンドマンのインタビュー
> [query] Total LLM token usage: 1045 tokens
> [query] Total embedding token usage: 45 tokens


'暗い学生時代から一転してスターとなったバンドマンのインタビュー'